In [ ]:
import hyperspy.api as hys
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt
import ipywidgets as pyw
import tkinter.filedialog as tkf

In [ ]:
%matplotlib inline

In [ ]:
eels_adr = tkf.askopenfilenames()
print(eels_adr)

In [ ]:
e_start = 1.0
e_end = 5.0
e_step = 0.01
energy_range = np.arange(e_start, e_end, e_step)
num_energy = len(energy_range)
print(num_energy)

In [ ]:
eels = []
for adr in eels_adr:
    temp = hys.load(adr, signal_type="EELS")
    print(temp)
    eels.append(temp)
print(len(eels))

In [ ]:
eels_cropped = []
eels_blurred = []
blur_sigma = 5
for spectrum in eels:
    temp = spectrum.isig[e_start:e_end].data
    eels_cropped.append(temp / np.max(temp))
    eels_blurred.append(ndimage.gaussian_filter(temp/np.max(temp), sigma=blur_sigma))
eels_cropped = np.asarray(eels_cropped)
print(eels_cropped.shape)
eels_blurred = np.asarray(eels_blurred)
print(eels_blurred.shape)

In [ ]:
eels_cropped = []
eels_blurred = []
slice_num = 2
blur_sigma = 5
for spectrum in eels:
    temp = spectrum[slice_num].isig[e_start:e_end].data
    eels_cropped.append(temp / np.max(temp))
    eels_blurred.append(ndimage.gaussian_filter(temp/np.max(temp), sigma=blur_sigma))
eels_cropped = np.asarray(eels_cropped)
print(eels_cropped.shape)
eels_blurred = np.asarray(eels_blurred)
print(eels_blurred.shape)

In [ ]:
color_rep = ["black", "gray", "green", "red", "yellow", "blue", "purple", "orange", "lime", "cyan", "magenta", "lightgray", "peru", "springgreen", "deepskyblue", "hotpink"]
print(len(color_rep))

In [ ]:
# Bandgap measurement of thin dielectric films using monochromated STEM-EELS
# Ultramicroscopy, Volume 109, Issue 9, August 2009, Pages 1183-1188
# https://doi.org/10.1016/j.ultramic.2009.04.005

%matplotlib qt
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

def bandgap_measurement(spectrum_ind, bg_range, slope_range):
    
    print("*************************************************")
    print("selected spectrum :")
    print(eels_adr[spectrum_ind])
    print("*************************************************")
    
    ax.cla()
    
    ax.plot(energy_range, eels_cropped[spectrum_ind], "k-")
    ax.set_ylim(np.min(eels_cropped[spectrum_ind]), np.max(eels_cropped[spectrum_ind]))
    ax.grid()
    
    bg_left = int(np.round((bg_range[0]-e_start)/e_step))
    #print(bg_left)
    bg_right = int(np.round((bg_range[1]-e_start)/e_step))
    slope_left = int(np.round((slope_range[0]-e_start)/e_step))
    slope_right = int(np.round((slope_range[1]-e_start)/e_step))
    
    ax.fill_between([energy_range[bg_left], energy_range[bg_right]], np.max(eels_cropped[spectrum_ind]), alpha=0.2, color="red")
    ax.fill_between([energy_range[slope_left], energy_range[slope_right]], np.max(eels_cropped[spectrum_ind]), alpha=0.2, color="blue")
    bg_flat = np.median(eels_cropped[spectrum_ind][bg_left:bg_right])
    #bg_flat = np.min(eels_cropped[spectrum_ind][bg_left:bg_right])
    ax.plot(energy_range, np.full_like(energy_range, bg_flat), "r--")
    
    poly_coeff = np.polyfit(energy_range[slope_left:slope_right], eels_cropped[spectrum_ind][slope_left:slope_right], deg=1)
    slope = poly_coeff[0]*energy_range + poly_coeff[1]
    ax.plot(energy_range, slope, "b--")
    
    intersection = (bg_flat - poly_coeff[1]) / poly_coeff[0]
    
    ax.plot(intersection, bg_flat, marker="o", color="black", label="Eg = %.2f eV"%intersection)
    #ax.set_xlim(-1.0, 14.0)
    #ax.set_ylim(0.000, 0.0015)
    ax.legend()
    fig.canvas.draw()
    fig.tight_layout()
    print("*************************************************")
    print("bandgap energy = %.2f"%intersection)
    print("*************************************************")

s_index = pyw.Dropdown(options=np.arange(len(eels_adr)), value=0, description="spectrum selection")
range_widget_1 = pyw.FloatRangeSlider(value=[1.0, 2.5], 
                                  min=energy_range[0], max=energy_range[-1], step=e_step, description="background range")
range_widget_2 = pyw.FloatRangeSlider(value=[3.5, 4.0], 
                                  min=energy_range[0], max=energy_range[-1], step=e_step, description="slope range")

pyw.interact(bandgap_measurement, spectrum_ind=s_index, bg_range=range_widget_1, slope_range=range_widget_2)
plt.show()

In [ ]:
# Bandgap measurement of thin dielectric films using monochromated STEM-EELS
# Ultramicroscopy, Volume 109, Issue 9, August 2009, Pages 1183-1188
# https://doi.org/10.1016/j.ultramic.2009.04.005

%matplotlib qt
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

def bandgap_measurement(spectrum_ind, bg_range, slope_range):
    
    print("*************************************************")
    print("selected spectrum :")
    print(eels_adr[spectrum_ind])
    print("*************************************************")
    
    ax.cla()
    
    ax.plot(energy_range, eels_blurred[spectrum_ind], "k-")
    ax.set_ylim(np.min(eels_blurred[spectrum_ind]), np.max(eels_blurred[spectrum_ind]))
    ax.grid()
    
    bg_left = int(np.round((bg_range[0]-e_start)/e_step))
    #print(bg_left)
    bg_right = int(np.round((bg_range[1]-e_start)/e_step))
    slope_left = int(np.round((slope_range[0]-e_start)/e_step))
    slope_right = int(np.round((slope_range[1]-e_start)/e_step))
    
    ax.fill_between([energy_range[bg_left], energy_range[bg_right]], np.max(eels_blurred[spectrum_ind]), alpha=0.2, color="red")
    ax.fill_between([energy_range[slope_left], energy_range[slope_right]], np.max(eels_blurred[spectrum_ind]), alpha=0.2, color="blue")
    bg_flat = np.median(eels_blurred[spectrum_ind][bg_left:bg_right])
    #bg_flat = np.min(eels_cropped[spectrum_ind][bg_left:bg_right])
    ax.plot(energy_range, np.full_like(energy_range, bg_flat), "r--")
    
    poly_coeff = np.polyfit(energy_range[slope_left:slope_right], eels_blurred[spectrum_ind][slope_left:slope_right], deg=1)
    slope = poly_coeff[0]*energy_range + poly_coeff[1]
    ax.plot(energy_range, slope, "b--")
    
    intersection = (bg_flat - poly_coeff[1]) / poly_coeff[0]
    
    ax.plot(intersection, bg_flat, marker="o", color="black", label="Eg = %.2f eV"%intersection)
    #ax.set_xlim(-1.0, 14.0)
    #ax.set_ylim(0.000, 0.0015)
    ax.legend()
    fig.canvas.draw()
    fig.tight_layout()
    print("*************************************************")
    print("bandgap energy = %.2f"%intersection)
    print("*************************************************")

s_index = pyw.Dropdown(options=np.arange(len(eels_adr)), value=0, description="spectrum selection")
range_widget_1 = pyw.FloatRangeSlider(value=[1.0, 2.5], 
                                  min=energy_range[0], max=energy_range[-1], step=e_step, description="background range")
range_widget_2 = pyw.FloatRangeSlider(value=[3.5, 4.0], 
                                  min=energy_range[0], max=energy_range[-1], step=e_step, description="slope range")

pyw.interact(bandgap_measurement, spectrum_ind=s_index, bg_range=range_widget_1, slope_range=range_widget_2)
plt.show()

In [ ]:
%matplotlib inline

In [ ]:
#%matplotlib qt
selected = [0, 1, 6, 7, 2, 3, 8, 9, 4, 5, 10, 11, 12]

fig, ax = plt.subplots(1, 1, figsize=(10, 15))
for t, i in enumerate(selected):
    ax.plot(energy_range, eels_cropped[i]+t*0.00005, color=color_rep[i+1])
    #temp = np.gradient(eels_cropped[i, 150:300], 30)
    #max_ind = np.argmax(temp) + 150
    #ax.plot(energy_range[max_ind], eels_cropped[i][max_ind]+i*0.15, "o", color=color_rep[i+1])
ax.set_xlim(-1.0, 12.0)
ax.set_ylim(0.00025, 0.0020)
ax.grid()
ax.tick_params(axis="both", labelsize=20)
ax.set_facecolor("black")
fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 15))
for i in range(len(eels_cropped)):
    ax.plot(energy_range, eels_cropped[i]+i*0.15, color=color_rep[i+1])
    #temp = np.gradient(eels_cropped[i, 150:300], 30)
    #max_ind = np.argmax(temp) + 150
    #ax.plot(energy_range[max_ind], eels_cropped[i][max_ind]+i*0.15, "o", color=color_rep[i+1])
ax.grid()
ax.tick_params(axis="both", labelsize=20)
ax.set_facecolor("black")
fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 15))
for i in range(len(eels_cropped)):
    ax.plot(energy_range, eels_blurred[i]+i*0.15, color=color_rep[i+1])
    #temp = np.gradient(eels_blurred[i, 150:400])
    #max_ind = np.argmax(temp) + 150
    #ax.plot(energy_range[max_ind], eels_blurred[i][max_ind]+i*0.15, "o", color=color_rep[i+1])
ax.grid()
ax.tick_params(axis="both", labelsize=20)
ax.set_facecolor("black")
fig.tight_layout()
plt.show()